# Evaluation with new Inference with limit of input seq length

In [1]:
import torch
import numpy as np
import data_visualization.Visualization as visualization

# Check if GPU is available, set device accordingly
device = torch.device("cuda" if torch.cuda.is_available() else 'cpu')

In [2]:
# Define Data parameters

sos_token = np.full((1, 24), 1)
pad_token = np.full((1, 24), 2)
pad_token = torch.tensor(pad_token, device=device)

dataset_dir = "/home/falaxdb/Repos/minus1/transformer_decoder_training/jupyter_notebooks/evaluation/specific_test_midis/split_into_mid"
snapshot_intervall = 0.05

batch_size = 1
seq_length = 2048
stride = seq_length

test_size=0.1

In [3]:
from transformer_decoder_training.dataprep_transformer.prepare_dataloader_complete import prepare_dataset_as_single_loader

# Load Data

#train_loader, val_loader, test_loader = prepare_dataset_as_dataloaders(dataset_dir, snapshot_intervall, batch_size, seq_length, stride, test_size, sos_token)
train_loader = prepare_dataset_as_single_loader(dataset_dir, snapshot_intervall, batch_size, seq_length, stride, sos_token, shuffle=False)

Processed dataset (2/2): 100%|██████████| 2/2 [00:00<00:00,  6.10it/s]

Processed 2 of 2 files


In [4]:
from transformer_decoder_training.transformer_inference_eval import inference_and_visualize_1

# Get sequence
sequence = next(iter(train_loader))
print(sequence.shape)

context_seq, continuing_seq, original_seq = inference_and_visualize_1.prepare_sequence(sequence, 513)

torch.Size([1, 2049, 24])


# 6.1

Model with BCELossWithlogitsloss

Load model

In [5]:
# Define model Parameters

# Embedding Size
hidden_size = 256
# Number of transformer blocks
num_layers = 8
# MultiheadAttention Heads
num_heads = 8

# Transformer without sigmoid output
from transformer_decoder_training.models.transformer_decoder_2 import Transformer

model = Transformer(num_emb=24, num_layers=num_layers, hidden_size=hidden_size, num_heads=num_heads).to(device)
model.load_state_dict(torch.load("/home/falaxdb/Repos/minus1/transformer_decoder_training/saved_files/saved_models/model_1_notebook_v6.1.pth"))
model.eval()

Transformer(
  (embedding): Linear(in_features=24, out_features=256, bias=True)
  (pos_emb): SinusoidalPosEmb()
  (blocks): ModuleList(
    (0-7): 8 x TransformerBlock(
      (norm1): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
      (multihead_attn): MultiheadAttention(
        (out_proj): NonDynamicallyQuantizableLinear(in_features=256, out_features=256, bias=True)
      )
      (norm2): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
      (mlp): Sequential(
        (0): Linear(in_features=256, out_features=1024, bias=True)
        (1): ELU(alpha=1.0)
        (2): Linear(in_features=1024, out_features=256, bias=True)
      )
    )
  )
  (fc_out): Linear(in_features=256, out_features=24, bias=True)
)

## Inference

In [6]:
midi_save_dir = "/home/falaxdb/Repos/minus1/transformer_decoder_training/saved_files/midi_outputs/notebook_6/6.1"

In [7]:
# inference with threshold and limited input seq
# limit = 513 Tokens

from transformer_decoder_training.inference import inference_5

tokens_with_truth, generated_logits = inference_5.inference(model, context_seq, continuing_seq, 0.17, pad_token, 513, device)

generated_sequence = inference_and_visualize_1.combine_output_with_context(tokens_with_truth, context_seq)

inference_and_visualize_1.inference_output_to_midi_one_octave(original_seq, context_seq, generated_sequence, 0.05, midi_save_dir, "limited_input_seq_513.mid")


Tokens to generate: 1536
Binary Tensors after concatinating:  torch.Size([1536, 24])
Binary Tensors after unsqueezing:  torch.Size([1, 1536, 24])
Context sequence + Generated tokens:  torch.Size([1, 2049, 24])
MIDI file saved to /home/falaxdb/Repos/minus1/transformer_decoder_training/saved_files/midi_outputs/notebook_6/6.1/limited_input_seq_513.mid


In [8]:
# inference with threshold and limited input seq
# limit = 1024 Tokens

from transformer_decoder_training.inference import inference_5

tokens_with_truth, generated_logits = inference_5.inference(model, context_seq, continuing_seq, 0.17, pad_token, 1024, device)

generated_sequence = inference_and_visualize_1.combine_output_with_context(tokens_with_truth, context_seq)

inference_and_visualize_1.inference_output_to_midi_one_octave(original_seq, context_seq, generated_sequence, 0.05, midi_save_dir, "limited_input_seq_1024.mid")

Tokens to generate: 1536
Binary Tensors after concatinating:  torch.Size([1536, 24])
Binary Tensors after unsqueezing:  torch.Size([1, 1536, 24])
Context sequence + Generated tokens:  torch.Size([1, 2049, 24])
MIDI file saved to /home/falaxdb/Repos/minus1/transformer_decoder_training/saved_files/midi_outputs/notebook_6/6.1/limited_input_seq_1024.mid


# 6.2

Model with basic CrossentropyLoss

In [9]:
# Define model Parameters

# Embedding Size
hidden_size = 256
# Number of transformer blocks
num_layers = 8
# MultiheadAttention Heads
num_heads = 8

# Transformer without sigmoid output
from transformer_decoder_training.models.transformer_decoder_2 import Transformer

model = Transformer(num_emb=24, num_layers=num_layers, hidden_size=hidden_size, num_heads=num_heads).to(device)
model.load_state_dict(torch.load("/home/falaxdb/Repos/minus1/transformer_decoder_training/saved_files/saved_models/model_1_notebook_v6.2.pth"))
model.eval()

Transformer(
  (embedding): Linear(in_features=24, out_features=256, bias=True)
  (pos_emb): SinusoidalPosEmb()
  (blocks): ModuleList(
    (0-7): 8 x TransformerBlock(
      (norm1): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
      (multihead_attn): MultiheadAttention(
        (out_proj): NonDynamicallyQuantizableLinear(in_features=256, out_features=256, bias=True)
      )
      (norm2): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
      (mlp): Sequential(
        (0): Linear(in_features=256, out_features=1024, bias=True)
        (1): ELU(alpha=1.0)
        (2): Linear(in_features=1024, out_features=256, bias=True)
      )
    )
  )
  (fc_out): Linear(in_features=256, out_features=24, bias=True)
)

In [10]:
midi_save_dir = "/home/falaxdb/Repos/minus1/transformer_decoder_training/saved_files/midi_outputs/notebook_6/6.2"

In [11]:
# inference with threshold and limited input seq
# limit = 513 Tokens

from transformer_decoder_training.inference import inference_5

tokens_with_truth, generated_logits = inference_5.inference(model, context_seq, continuing_seq, 0.999, pad_token, 513, device)

generated_sequence = inference_and_visualize_1.combine_output_with_context(tokens_with_truth, context_seq)

inference_and_visualize_1.inference_output_to_midi_one_octave(original_seq, context_seq, generated_sequence, 0.05, midi_save_dir, "limited_input_seq_513.mid")

Tokens to generate: 1536
Binary Tensors after concatinating:  torch.Size([1536, 24])
Binary Tensors after unsqueezing:  torch.Size([1, 1536, 24])
Context sequence + Generated tokens:  torch.Size([1, 2049, 24])
MIDI file saved to /home/falaxdb/Repos/minus1/transformer_decoder_training/saved_files/midi_outputs/notebook_6/6.2/limited_input_seq_513.mid


In [12]:
# inference with threshold and limited input seq
# limit = 1024 Tokens

from transformer_decoder_training.inference import inference_5

tokens_with_truth, generated_logits = inference_5.inference(model, context_seq, continuing_seq, 0.999, pad_token, 1024, device)

generated_sequence = inference_and_visualize_1.combine_output_with_context(tokens_with_truth, context_seq)

inference_and_visualize_1.inference_output_to_midi_one_octave(original_seq, context_seq, generated_sequence, 0.05, midi_save_dir, "limited_input_seq_1024.mid")

Tokens to generate: 1536
Binary Tensors after concatinating:  torch.Size([1536, 24])
Binary Tensors after unsqueezing:  torch.Size([1, 1536, 24])
Context sequence + Generated tokens:  torch.Size([1, 2049, 24])
MIDI file saved to /home/falaxdb/Repos/minus1/transformer_decoder_training/saved_files/midi_outputs/notebook_6/6.2/limited_input_seq_1024.mid


# 6.3

Model with melody + Harmony as input and Harmony only as output

In [13]:
# Define model Parameters

# Embedding Size
hidden_size = 256
# Number of transformer blocks
num_layers = 8
# MultiheadAttention Heads
num_heads = 8

# Transformer without sigmoid output
from transformer_decoder_training.models.transformer_decoder_2 import Transformer

model = Transformer(num_emb=24, num_layers=num_layers, hidden_size=hidden_size, num_heads=num_heads, num_output_dim=12).to(device)
model.load_state_dict(torch.load("/home/falaxdb/Repos/minus1/transformer_decoder_training/saved_files/saved_models/model_1_notebook_v6.3.pth"))
model.eval()

Transformer(
  (embedding): Linear(in_features=24, out_features=256, bias=True)
  (pos_emb): SinusoidalPosEmb()
  (blocks): ModuleList(
    (0-7): 8 x TransformerBlock(
      (norm1): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
      (multihead_attn): MultiheadAttention(
        (out_proj): NonDynamicallyQuantizableLinear(in_features=256, out_features=256, bias=True)
      )
      (norm2): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
      (mlp): Sequential(
        (0): Linear(in_features=256, out_features=1024, bias=True)
        (1): ELU(alpha=1.0)
        (2): Linear(in_features=1024, out_features=256, bias=True)
      )
    )
  )
  (fc_out): Linear(in_features=256, out_features=12, bias=True)
)

In [14]:
midi_save_dir = "/home/falaxdb/Repos/minus1/transformer_decoder_training/saved_files/midi_outputs/notebook_6/6.3"

In [15]:
# inference with threshold and limited input seq
# limit = 513 Tokens

from transformer_decoder_training.inference import inference_5

tokens_with_truth, generated_logits = inference_5.inference_melody_only(model, context_seq, continuing_seq, 0.17, pad_token, 513, device)

print("last generated token: ", tokens_with_truth[-1].shape)

generated_sequence = inference_and_visualize_1.combine_output_with_context(tokens_with_truth, context_seq)

inference_and_visualize_1.inference_output_to_midi_one_octave(original_seq, context_seq, generated_sequence, 0.05, midi_save_dir, "limited_input_seq_513.mid")

Tokens to generate: 1536
Next Token shape: torch.Size([1, 12]), Ground truth shape: torch.Size([1, 12])
Next Token shape: torch.Size([1, 12]), Ground truth shape: torch.Size([1, 12])
Next Token shape: torch.Size([1, 12]), Ground truth shape: torch.Size([1, 12])
Next Token shape: torch.Size([1, 12]), Ground truth shape: torch.Size([1, 12])
Next Token shape: torch.Size([1, 12]), Ground truth shape: torch.Size([1, 12])
Next Token shape: torch.Size([1, 12]), Ground truth shape: torch.Size([1, 12])
Next Token shape: torch.Size([1, 12]), Ground truth shape: torch.Size([1, 12])
Next Token shape: torch.Size([1, 12]), Ground truth shape: torch.Size([1, 12])
Next Token shape: torch.Size([1, 12]), Ground truth shape: torch.Size([1, 12])
Next Token shape: torch.Size([1, 12]), Ground truth shape: torch.Size([1, 12])
Next Token shape: torch.Size([1, 12]), Ground truth shape: torch.Size([1, 12])
Next Token shape: torch.Size([1, 12]), Ground truth shape: torch.Size([1, 12])
Next Token shape: torch.Siz